In [1]:
%load_ext autoreload

In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle

from enterprise import constants as const
from enterprise.signals import parameter
from enterprise.signals import selections
from enterprise.signals import signal_base
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
from enterprise.signals import utils

from utils import models
from utils import hypermod
from utils.sample_helpers import JumpProposal, get_parameter_groups

from utils import sample_utils as su

from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
from acor import acor

%matplotlib inline
%autoreload 2

# Read in data

In [3]:
ephem = 'DE436'
datadir = '/home/pbaker/nanograv/data/'

In [4]:
# read in data pickles
filename = datadir + 'nano11_{}.pkl'.format(ephem)
with open(filename, "rb") as f:
    psrs = pickle.load(f)

filename = datadir + 'nano11_setpars.pkl'
with open(filename, "rb") as f:
    noise_dict = pickle.load(f)

In [5]:
psr_11yr = [ # the NG 11yr pulsars
    'B1855+09',
    'B1937+21',
    'B1953+29',
    'J1713+0747',
    'J1909-3744',
    'J2317+1439',
    'J1600-3053',
    'J1640+2224',
    'J1614-2230',
    'J1918-0642',
    'J2043+1711',
    'J0030+0451',
    'J1744-1134',
    'J1741+1351',
    'J0613-0200',
    'J2010-1323',
    'J1455-3330',
    'J0645+5158',
    'J1012+5307',
    'J2145-0750',
    'J1024-0719',
    'J1738+0333',
    'J1910+1256',
    'J1923+2515',
    'J1853+1303',
    'J1944+0907',
    'J2017+0603',
    'J1643-1224',
    'J0340+4130',
    'J2214+3000',
    'J1903+0327',
    'J1747-4036',
    'J2302+4442',
    'J0023+0923',
]

psr_9yr = [ # the NG 9yr pulsars
    'J1713+0747',
    'J1909-3744',
    'J1640+2224',
    'J1600-3053',
    'J2317+1439',
    'J1918-0642',
    'J1614-2230',
    'J1744-1134',
    'J0030+0451',
    'J2145-0750',
    'B1855+09',
    'J1853+1303',
    'J0613-0200',
    'J1455-3330',
    'J1741+1351',
    'J2010-1323',
    'J1024-0719',
    'J1012+5307',
]

psrs = [p for p in psrs if p.name in psr_11yr]

In [6]:
len(psrs)

34

# setup models

## custom BWM w/ k param

In [7]:
@signal_base.function
def bwm_delay(toas, pos, log10_h=-14.0, cos_gwtheta=0.0, gwphi=0.0,
              gwpol=0.0, t0=55000, psrk=1, antenna_pattern_fn=None):
    """
    Function that calculates the earth-term gravitational-wave
    burst-with-memory signal, as described in:
    Seto et al, van haasteren and Levin, phsirkov et al, Cordes and Jenet.
    This version uses the F+/Fx polarization modes, as verified with the
    Continuous Wave and Anisotropy papers.
    :param toas: Time-of-arrival measurements [s]
    :param pos: Unit vector from Earth to pulsar
    :param log10_h: log10 of GW strain
    :param cos_gwtheta: Cosine of GW polar angle
    :param gwphi: GW azimuthal polar angle [rad]
    :param gwpol: GW polarization angle
    :param t0: Burst central time [day]
    :param antenna_pattern_fn:
        User defined function that takes `pos`, `gwtheta`, `gwphi` as
        arguments and returns (fplus, fcross)
    :return: the waveform as induced timing residuals (seconds)
    """

    # convert
    h = 10**log10_h
    gwtheta = np.arccos(cos_gwtheta)
    t0 *= const.day

    # antenna patterns
    if antenna_pattern_fn is None:
        apc = utils.create_gw_antenna_pattern(pos, gwtheta, gwphi)
    else:
        apc = antenna_pattern_fn(pos, gwtheta, gwphi)

    # grab fplus, fcross
    fp, fc = apc[0], apc[1]

    # combined polarization
    pol = np.cos(2*gwpol)*fp + np.sin(2*gwpol)*fc

    # Define the heaviside function
    heaviside = lambda x: 0.5 * (np.sign(x) + 1)

    k = np.rint(psrk)
    
    # Return the time-series for the pulsar
    return k * pol * h * heaviside(toas-t0) * (toas-t0)

## build PTA

In [8]:
outdir = '/home/pbaker/nanograv/bwm/dropout_11/'
!mkdir -p $outdir

In [9]:
# find the maximum time span to set frequency sampling
tmin = np.min([p.toas.min() for p in psrs])
tmax = np.max([p.toas.max() for p in psrs])
Tspan = tmax - tmin
print("Tspan = {:f} sec ~ {:.2f} yr".format(Tspan, Tspan/const.yr))

# find clipped prior range for bwm_t0
clip = 0.05 * Tspan
t0min = (tmin + 2*clip)/const.day  # don't search in first 10%
t0max = (tmax - clip)/const.day  # don't search in last 5%
print("search for t0 in [{:.1f}, {:.1f}] MJD".format(t0min, t0max))

Tspan = 360431065.460732 sec ~ 11.42 yr
search for t0 in [53633.3, 57179.2] MJD


In [10]:
anomaly = {
    'bwm_costheta':0.10,
    'bwm_log10_A':-12.77,
    'bwm_phi':1.15,
    'bwm_pol':2.81,
    'bwm_t0':55421.59,
}

In [11]:
# White Noise
selection = selections.Selection(selections.by_backend)

efac = parameter.Constant()
equad = parameter.Constant()
ecorr = parameter.Constant()

ef = white_signals.MeasurementNoise(efac=efac, selection=selection)
eq = white_signals.EquadNoise(log10_equad=equad, selection=selection)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection)

wn = ef + eq + ec


# Red Noise
rn_log10_A = parameter.Uniform(-20, -11)
rn_gamma = parameter.Uniform(0, 7)
rn_powlaw = utils.powerlaw(log10_A=rn_log10_A, gamma=rn_gamma)
rn = gp_signals.FourierBasisGP(rn_powlaw, components=30, Tspan=Tspan)


# BWM signal
name = 'bwm'
amp_name = '{}_log10_A'.format(name)
bwm_log10_A = parameter.Uniform(-18, -12)(amp_name)

pol_name = '{}_pol'.format(name)
pol = parameter.Constant(anomaly[pol_name])(pol_name)

t0_name = '{}_t0'.format(name)
t0 = parameter.Constant(anomaly[t0_name])(t0_name)

costh_name = '{}_costheta'.format(name)
phi_name = '{}_phi'.format(name)
costh = parameter.Constant(anomaly[costh_name])(costh_name)
phi = parameter.Constant(anomaly[phi_name])(phi_name)

k = parameter.Uniform(0,1) # not common, one per PSR
bwm_wf = bwm_delay(log10_h=bwm_log10_A, t0=t0,
                   cos_gwtheta=costh, gwphi=phi, gwpol=pol,
                   psrk=k)
bwm = deterministic_signals.Deterministic(bwm_wf, name=name)


# Timing Model
tm = gp_signals.TimingModel(use_svd=True)

In [12]:
mod = tm + wn + rn + bwm

pta = signal_base.PTA([mod(psr) for psr in psrs])
pta.set_default_params(noise_dict)

INFO: enterprise.signals.signal_base: Setting B1855+09_430_ASP_efac to 1.15823
INFO: enterprise.signals.signal_base: Setting B1855+09_430_PUPPI_efac to 1.12256
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_ASP_efac to 1.08416
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_PUPPI_efac to 1.37942
INFO: enterprise.signals.signal_base: Setting B1855+09_430_ASP_log10_equad to -7.56768
INFO: enterprise.signals.signal_base: Setting B1855+09_430_PUPPI_log10_equad to -6.17493
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_ASP_log10_equad to -6.5087
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_PUPPI_log10_equad to -6.53584
INFO: enterprise.signals.signal_base: Setting B1855+09_430_ASP_log10_ecorr to -8.15092
INFO: enterprise.signals.signal_base: Setting B1855+09_430_PUPPI_log10_ecorr to -6.29747
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_ASP_log10_ecorr to -6.09581
INFO: enterprise.signals.signal_base: Setting B1855

INFO: enterprise.signals.signal_base: Setting J0645+5158_Rcvr_800_GUPPI_log10_equad to -7.0759
INFO: enterprise.signals.signal_base: Setting J0645+5158_Rcvr1_2_GUPPI_log10_ecorr to -7.83345
INFO: enterprise.signals.signal_base: Setting J0645+5158_Rcvr_800_GUPPI_log10_ecorr to -7.15591
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr1_2_GASP_efac to 1.05466
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr1_2_GUPPI_efac to 1.0801
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr_800_GASP_efac to 1.14312
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr_800_GUPPI_efac to 1.1831
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr1_2_GASP_log10_equad to -6.54824
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr1_2_GUPPI_log10_equad to -6.49869
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr_800_GASP_log10_equad to -6.67014
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr_800_GUPPI_log10_equad 

INFO: enterprise.signals.signal_base: Setting J1713+0747_Rcvr1_2_GUPPI_efac to 1.03958
INFO: enterprise.signals.signal_base: Setting J1713+0747_Rcvr_800_GASP_efac to 1.13148
INFO: enterprise.signals.signal_base: Setting J1713+0747_Rcvr_800_GUPPI_efac to 1.06597
INFO: enterprise.signals.signal_base: Setting J1713+0747_S-wide_ASP_efac to 1.12037
INFO: enterprise.signals.signal_base: Setting J1713+0747_S-wide_PUPPI_efac to 1.1154
INFO: enterprise.signals.signal_base: Setting J1713+0747_L-wide_ASP_log10_equad to -7.55444
INFO: enterprise.signals.signal_base: Setting J1713+0747_L-wide_PUPPI_log10_equad to -7.90314
INFO: enterprise.signals.signal_base: Setting J1713+0747_Rcvr1_2_GASP_log10_equad to -7.31355
INFO: enterprise.signals.signal_base: Setting J1713+0747_Rcvr1_2_GUPPI_log10_equad to -8.03232
INFO: enterprise.signals.signal_base: Setting J1713+0747_Rcvr_800_GASP_log10_equad to -6.94987
INFO: enterprise.signals.signal_base: Setting J1713+0747_Rcvr_800_GUPPI_log10_equad to -7.13031
INF

INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GUPPI_efac to 1.051
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GASP_efac to 0.989049
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GUPPI_efac to 1.04172
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GASP_log10_equad to -7.44335
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GUPPI_log10_equad to -8.07509
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GASP_log10_equad to -6.62237
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GUPPI_log10_equad to -7.33716
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GASP_log10_ecorr to -8.47751
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GUPPI_log10_ecorr to -7.12103
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GASP_log10_ecorr to -7.92153
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GUPPI_log

INFO: enterprise.signals.signal_base: Setting J2043+1711_L-wide_ASP_log10_ecorr to -7.51748
INFO: enterprise.signals.signal_base: Setting J2043+1711_L-wide_PUPPI_log10_ecorr to -8.48264
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GASP_efac to 0.981934
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GUPPI_efac to 1.1003
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr_800_GASP_efac to 1.40428
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr_800_GUPPI_efac to 1.13029
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GASP_log10_equad to -5.69371
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GUPPI_log10_equad to -6.3098
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr_800_GASP_log10_equad to -6.51163
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr_800_GUPPI_log10_equad to -6.28187
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GASP_log10_ecorr to -6

In [13]:
summary = pta.summary()
print(summary)

Signal Name                              Signal Class                   no. Parameters      
B1855+09_linear_timing_model_svd         TimingModel                    0                   

params:
__________________________________________________________________________________________
B1855+09_efac                            MeasurementNoise               0                   

params:
B1855+09_430_ASP_efac:Constant=1.15823                                                    
B1855+09_430_PUPPI_efac:Constant=1.12256                                                  
B1855+09_L-wide_ASP_efac:Constant=1.08416                                                 
B1855+09_L-wide_PUPPI_efac:Constant=1.37942                                               
__________________________________________________________________________________________
B1855+09_equad                           EquadNoise                     0                   

params:
B1855+09_430_ASP_log10_equad:Constant=-7.56768         

# Sample

## initial point and covariance matrix

In [14]:
x0 = np.hstack([noise_dict[p.name] if p.name in noise_dict.keys()
                else p.sample() for p in pta.params])  # initial point
ndim = len(x0)

# initial jump covariance matrix
# set initial cov stdev to (starting order of magnitude)/10
stdev = np.array([10**np.floor(np.log10(abs(x)))/10 for x in x0])
cov = np.diag(stdev**2)

## sampling groups

In [15]:
# generate custom sampling groups
groups = [list(range(ndim))]  # all params

# pulsar noise groups (RN)
for psr in psrs:
    this_group = [pta.param_names.index(par)
                  for par in pta.param_names if psr.name in par]
    groups.append(this_group)

# all k params
this_group = []
for par in pta.param_names:
    if '_psrk' in par:
        this_group.append(pta.param_names.index(par))
groups.append(this_group)

# bwm params
this_group = [pta.param_names.index('bwm_log10_A')]
for ii in range(2):
    groups.append(this_group)
    
this_group = [pta.param_names.index(par)
              for par in pta.param_names if 'bwm_' in par]
groups.append(this_group)

## initialize sampler object

In [16]:
sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov, groups=groups,
                 outDir=outdir, resume=False)

In [17]:
sumfile = os.path.join(outdir, 'summary.txt')
with open(sumfile, 'w') as f:
    f.write(pta.summary())

outfile = os.path.join(outdir, 'params.txt')
with open(outfile, 'w') as f:
    for pname in pta.param_names:
        f.write(pname+'\n')

In [18]:
# additional proposals
full_prior = su.build_prior_draw(pta, pta.param_names, name='full_prior')
sampler.addProposalToCycle(full_prior, 5)

# RN empirical
#from utils.sample_utils import EmpiricalDistribution2D
#with open("/home/pbaker/nanograv/data/nano11_RNdistr.pkl", "rb") as f:
#    distr = pickle.load(f)
#Non4 = len(distr) // 4
#RN_emp = su.EmpDistrDraw(distr, pta.param_names, Nmax=Non4, name='RN_empirical')
#sampler.addProposalToCycle(RN_emp, 10)

## sample it!

In [ ]:
thin = 50
Nsamp = 100000 * 50
sampler.sample(x0, Nsamp,
               SCAMweight=30, AMweight=20, DEweight=50,
               burn=int(5e4), thin=thin)

/home/pbaker/.local/opt/miniconda/envs/enterprise/lib/python3.6/site-packages/enterprise/signals/parameter.py:66: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))


Finished 1.00 percent in 6639.887784 s Acceptance rate = 0.509242Adding DE jump with weight 50
Finished 2.10 percent in 14041.509329 s Acceptance rate = 0.474019